In [0]:
spark.conf.get("spark.databricks.clusterUsageTags.clusterName")
spark.conf.get("spark.databricks.clusterUsageTags.clusterId")

'0422-041613-7jba4tw0'

In [0]:
#Connections credentials
ContainerName = "output"
azure_blobstorage_name = "teamdsde"
mountpointname = "/mnt/rfm"
secret_key ="l5wIsV649yQmqL4Oe1sOGLlPGlQq8mJHitaqjydN2qOtUkBbYUXiJrEXqYnrTuCZSDJ8Wip5HA3E+AStYEpsgA=="

In [0]:
#mount
#dbutils.fs.mount(source = f"wasbs://output@teamdsde.blob.core.windows.net",mount_point = mountpointname ,#extra_configs = {"fs.azure.account.key."+azure_blobstorage_name+".blob.core.windows.net":secret_key})
     

In [0]:

DF = spark.read.format('csv').options(
    header='true', inferschema='true').load("/mnt/rfm/*.csv")

In [0]:
rowscount=DF.count()
colcount=len(DF.columns)
print("Dataframe rows:",rowscount)
print("Dataframe cols:",colcount)

Dataframe rows: 485123
Dataframe cols: 8


In [0]:
#Null count
from pyspark.sql.functions import col,isnan, when, count
DF.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in DF.columns]
   ).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|          0|       0|          0|        0|    130802|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [0]:
DF1=DF.na.drop(subset=["CustomerID"])
DF1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in DF1.columns]
   ).show()
     

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|          0|       0|          0|        0|         0|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [0]:

DF1.printSchema()
DF1.show(10)
     

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|  

In [0]:
#Imports
# #def square(x):  
#    return x**2  

import pandas as pd
pandasDF = DF1.toPandas()
pandasDF['InvoiceDate']= pd.to_datetime(pandasDF['InvoiceDate'])
pandasDF["Date"]=pd.DatetimeIndex(pandasDF["InvoiceDate"]).date
pandasDF["Month"]=pandasDF["InvoiceDate"].dt.month
pandasDF["Year"]=pandasDF["InvoiceDate"].dt.year
DF2 = spark.createDataFrame(pandasDF)
DF2.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----------+-----+----+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|      Date|Month|Year|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----------+-----+----+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|     17850|United Kingdom|2010-12-01|   12|2010|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|2010-12-01|   12|2010|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|     17850|United Kingdom|2010-12-01|   12|2010|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|2010-12-01|   12|2010|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|     1785

In [0]:

from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.types import *
DF3=DF2.withColumn("Totalsales",col("UnitPrice")*col("Quantity"))
Totalsalesbymonth=DF3.groupBy('Description').sum('Totalsales')
Totalsalesbymonth.show()

+--------------------+------------------+
|         Description|   sum(Totalsales)|
+--------------------+------------------+
|10 COLOUR SPACEBO...|3669.0899999999992|
|SET/10 BLUE POLKA...|            2362.5|
|POTTING SHED SOW ...|              4.25|
|SET/3 RED GINGHAM...| 8048.550000000001|
|MAGNETS PACK OF 4...|453.25000000000006|
|WHITE CHRYSANTHEM...| 79.80000000000001|
|SILVER FABRIC MIRROR|157.70000000000005|
|PINK  HONEYCOMB P...|             238.3|
|PAPERWEIGHT KINGS...|             165.3|
|PINK BOUDOIR T-LI...|              10.2|
|GLASS CAKE COVER ...|              29.9|
| BLACK CHERRY LIGHTS|              54.0|
|WOVEN BERRIES CUS...|             74.25|
|IVORY ENCHANTED F...| 546.8999999999999|
|PACK OF 12 COLOUR...|1862.7999999999997|
|ANTIQUE SILVER TE...|4003.0000000000014|
|   CLAM SHELL SMALL |            371.53|
|TWO DOOR CURIO CA...|2398.9500000000003|
|MULTICOLOUR HONEY...|331.67999999999995|
|GREEN GOOSE FEATH...| 361.3500000000001|
+--------------------+------------

In [0]:

date_max = DF3.select(F.max(DF3.InvoiceDate).alias('max_date')).toPandas()
DF4 = DF3.withColumn('Duration', F.datediff(F.lit(date_max.iloc[0][0]), 'InvoiceDate')) #'2011-12-04 13:15:00'
DF4.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----------+-----+----+------------------+--------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|      Date|Month|Year|        Totalsales|Duration|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----------+-----+----+------------------+--------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|     17850|United Kingdom|2010-12-01|   12|2010|15.299999999999999|     373|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|2010-12-01|   12|2010|             20.34|     373|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|     17850|United Kingdom|2010-12-01|   12|2010|              22.0|     373|
|   536365|   84029G|KNITTED UNION FLA...|       6|2

In [0]:
recency = DF4.groupby('CustomerID').agg(F.min('Duration').alias('Recency'))
recency.sort(col('Recency')).show()

+----------+-------+
|CustomerID|Recency|
+----------+-------+
|     14422|      0|
|     17754|      0|
|     16954|      0|
|     17001|      0|
|     13426|      0|
|     17389|      0|
|     18102|      0|
|     14051|      0|
|     16558|      0|
|     15311|      0|
|     16626|      0|
|     17428|      0|
|     15910|      0|
|     13069|      0|
|     13113|      0|
|     14441|      0|
|     17364|      0|
|     13777|      0|
|     12748|      0|
|     16705|      0|
+----------+-------+
only showing top 20 rows



In [0]:
frequency = DF4.groupby('CustomerID', 'InvoiceDate').count()\
                        .groupby('CustomerID')\
                        .agg(F.count("*").alias("Frequency"))
frequency.orderBy(frequency.Frequency.desc()).show(5)

+----------+---------+
|CustomerID|Frequency|
+----------+---------+
|     12748|      210|
|     17841|      124|
|     14606|       95|
|     13089|       93|
|     15311|       91|
+----------+---------+
only showing top 5 rows



In [0]:

monetary = DF4.groupby('CustomerID').agg(F.round(F.sum('Totalsales'), 2).alias('Monetary_value'))
monetary.orderBy(F.col('Monetary_value').desc()).show(5)

+----------+--------------+
|CustomerID|Monetary_value|
+----------+--------------+
|     18102|      259657.3|
|     17450|     194550.79|
|     16446|      168472.5|
|     17511|      91062.38|
|     16029|      81024.84|
+----------+--------------+
only showing top 5 rows



In [0]:
rfm = recency.join(frequency, on = 'CustomerID', how = 'inner')\
             .join(monetary, on = 'CustomerID', how = 'inner')
rfm.show(5)

+----------+-------+---------+--------------+
|CustomerID|Recency|Frequency|Monetary_value|
+----------+-------+---------+--------------+
|     15727|     16|        7|       5178.96|
|     17389|      0|       34|      31833.68|
|     16503|    106|        4|       1431.93|
|     14570|    280|        2|        218.06|
|     17420|     50|        3|        598.83|
+----------+-------+---------+--------------+
only showing top 5 rows



In [0]:

rfm.where(rfm.Recency.contains(0)).show()

+----------+-------+---------+--------------+
|CustomerID|Recency|Frequency|Monetary_value|
+----------+-------+---------+--------------+
|     17389|      0|       34|      31833.68|
|     16503|    106|        4|       1431.93|
|     14570|    280|        2|        218.06|
|     17420|     50|        3|        598.83|
|     13623|     30|        5|        747.78|
|     15447|    330|        1|        155.17|
|     14450|    180|        3|        483.25|
|     15790|     10|        1|        220.85|
|     15619|     10|        1|         336.4|
|     15296|     50|        6|       3672.86|
|     17754|      0|        5|       1772.26|
|     17223|    310|        2|        426.79|
|     14779|    280|        3|        386.15|
|     15292|    201|        1|        180.94|
|     15026|     80|        3|         469.0|
|     16549|     10|       10|       4154.64|
|     17213|     40|        7|       2781.23|
|     14284|     30|        2|        595.25|
|     16601|     40|        1|    

In [0]:
def r_score(r_value):
    """recency with lower value ranks the least, i.e the 
    more recent a user transaction, the lower the rank"""
    
    if r_value <= 14:    # 2 weeks
        return 1
    elif r_value <= 31:   # 1 month
        return 2
    elif r_value <= 93:    # 3 month
        return 3
    else:
        return 
    
def f_score(f_value):
    """the more a customer transact with the business, 
    the more their rank is closer to 1, the fewer the number of 
    times they have made transaction, the higher the rank"""
    
    if f_value <= 3:
        return 4
    elif f_value <= 18:
        return 3
    elif f_value <= 36:
        return 2
    else:
        return 1

    
def m_score(m_value):
    """the more the value contributed to the business, 
    the higher their rank to 1"""
    
    if m_value < 10:
        return 4
    elif m_value <= 100:
        return 3
    elif m_value <= 1000:
        return 2
    else:
        return 1
    
r_udf = F.udf(lambda r_value: r_score(r_value), StringType())
f_udf = F.udf(lambda f_value: f_score(f_value), StringType())pp
m_udf = F.udf(lambda m_value: m_score(m_value), StringType())

In [0]:

rfm_seg = rfm.withColumn('r_score', r_udf(F.col('recency')))
rfm_seg = rfm_seg.withColumn('f_score', f_udf(F.col('frequency')))
rfm_seg = rfm_seg.withColumn('m_score', m_udf(F.col('monetary_value')))
rfm_seg.show(5)

+----------+-------+---------+--------------+-------+-------+-------+
|CustomerID|Recency|Frequency|Monetary_value|r_score|f_score|m_score|
+----------+-------+---------+--------------+-------+-------+-------+
|     15727|     16|        7|       5178.96|      2|      3|      1|
|     17389|      0|       34|      31833.68|      1|      2|      1|
|     16503|    106|        4|       1431.93|   NULL|      3|      1|
|     14570|    280|        2|        218.06|   NULL|      4|      2|
|     17420|     50|        3|        598.83|      3|      4|      2|
+----------+-------+---------+--------------+-------+-------+-------+
only showing top 5 rows



In [0]:

rowscount=rfm_seg.count()
colcount=len(rfm_seg.columns)
print("Dataframe rows:",rowscount)
print("Dataframe cols:",colcount)

Dataframe rows: 3920
Dataframe cols: 7


In [0]:

rfm_seg.write.mode("overwrite").format("csv").save("/mnt/RFMdata/RFMresults/RFMscores.csv")
     